In [9]:
# Parâmetros do pipeline
input_config_path = 'config.json'
input_validacao_path = 'validacao_resultado.json'
output_metricas_path = 'metricas_completas.csv'

In [10]:
import boto3
import json
import pandas as pd
from datetime import datetime
import os

In [11]:
print(f"Início do cálculo: {datetime.now()}")
print(f"Configuração: {input_config_path}")
print(f"Validação: {input_validacao_path}")
print(f"Saída: {output_metricas_path}")

Início do cálculo: 2025-07-22 15:11:32.133995
Configuração: config.json
Validação: validacao_resultado.json
Saída: metricas_completas.csv


In [12]:
s3 = boto3.client('s3')

# Baixar config.json
s3.download_file('experimento-lucas-barbosa', f'config/{input_config_path}', 'config.json')
with open('config.json', 'r') as f:
    config = json.load(f)

print(f"Configuração carregada: {config}")

# Baixar validacao_resultado.json
s3.download_file('experimento-lucas-barbosa', f'artifacts/{input_validacao_path}', 'validacao_resultado.json')
with open('validacao_resultado.json', 'r') as f:
    validacao = json.load(f)

print(f"Validação carregada: {len(validacao.get('arquivos_validos', []))} arquivos válidos")

# Extrair variáveis do config para uso
bucket_name = config['bucket_name']
pasta_raw = config['pasta_raw']
pasta_processed = config['pasta_processed']
grupo_controle = config['grupo_controle']
colunas_obrigatorias = config.get('colunas_obrigatorias', ['id_experimento', 'id_customer', 'grupo', 'uso', 'valor', 'timestamp'])

print(f"Bucket: {bucket_name}")
print(f"Grupo controle: {grupo_controle}")

Configuração carregada: {'bucket_name': 'experimento-lucas-barbosa', 'pasta_raw': 'raw/', 'pasta_processed': 'processed/', 'pasta_archive': 'archive/', 'colunas_obrigatorias': ['id_experimento', 'id_customer', 'grupo', 'uso', 'valor', 'timestamp'], 'grupo_controle': 'GC'}
Validação carregada: 0 arquivos válidos
Bucket: experimento-lucas-barbosa
Grupo controle: GC


In [13]:
# --- Funções de Cálculo de Métricas --- 

def calcular_metricas_basicas(df_grupo: pd.DataFrame, id_experimento: str, grupo: str, arquivo_origem: str) -> list:
    """Calcula métricas básicas para um grupo específico."""
    metricas = []
    timestamp = datetime.now().isoformat()
    total_usuarios = len(df_grupo)

    # Taxa de Conversão
    conversao = (df_grupo['uso'] > 0).sum() / total_usuarios * 100 if total_usuarios else 0
    metricas.append({'id_experimento': id_experimento, 'grupo': grupo, 'metrica': 'taxa_conversao', 'valor': round(conversao, 2), 'timestamp': timestamp, 'arquivo_origem': arquivo_origem})

    # Uso Médio
    uso_medio = df_grupo['uso'].mean() if total_usuarios else 0
    metricas.append({'id_experimento': id_experimento, 'grupo': grupo, 'metrica': 'uso_medio', 'valor': round(uso_medio, 2), 'timestamp': timestamp, 'arquivo_origem': arquivo_origem})

    # Valor Médio
    valor_medio = df_grupo['valor'].mean() if total_usuarios else 0
    metricas.append({'id_experimento': id_experimento, 'grupo': grupo, 'metrica': 'valor_medio', 'valor': round(valor_medio, 2), 'timestamp': timestamp, 'arquivo_origem': arquivo_origem})
    
    # Número de Usuários Únicos
    usuarios_unicos = df_grupo['id_customer'].nunique() if 'id_customer' in df_grupo.columns else total_usuarios
    metricas.append({'id_experimento': id_experimento, 'grupo': grupo, 'metrica': 'usuarios_unicos', 'valor': usuarios_unicos, 'timestamp': timestamp, 'arquivo_origem': arquivo_origem})

    # --- Adicione novas métricas básicas aqui! --- 
    # Exemplo: Valor Total
    # valor_total = df_grupo['valor'].sum() if total_usuarios else 0
    # metricas.append({'id_experimento': id_experimento, 'grupo': grupo, 'metrica': 'valor_total', 'valor': round(valor_total, 2), 'timestamp': timestamp, 'arquivo_origem': arquivo_origem})

    return metricas

def calcular_lifts(df_metricas_basicas: pd.DataFrame, id_experimento: str, grupo_controle: str, arquivo_origem: str) -> list:
    """Calcula métricas de lift comparando grupos com o grupo de controle."""
    lifts = []
    timestamp = datetime.now().isoformat()

    df_gc = df_metricas_basicas[df_metricas_basicas['grupo'] == grupo_controle]

    for metrica_base in ['taxa_conversao', 'uso_medio', 'valor_medio', 'usuarios_unicos']:
        base_rows = df_gc[df_gc['metrica'] == metrica_base]
        if base_rows.empty:
            print(f" Métrica base '{metrica_base}' não encontrada para o grupo de controle. Pulando lifts.")
            continue
        valor_controle = base_rows['valor'].iloc[0]

        if valor_controle <= 0:
            print(f" Valor de controle para '{metrica_base}' é zero ou negativo. Pulando lifts.")
            continue

        for grupo in df_metricas_basicas['grupo'].unique():
            if grupo == grupo_controle: continue

            grupo_rows = df_metricas_basicas[(df_metricas_basicas['grupo'] == grupo) & (df_metricas_basicas['metrica'] == metrica_base)]
            if grupo_rows.empty:
                print(f" Métrica '{metrica_base}' não encontrada para o grupo '{grupo}'. Pulando lift.")
                continue
            valor_grupo = grupo_rows['valor'].iloc[0]

            lift_valor = round((valor_grupo - valor_controle) / valor_controle * 100, 2)
            lifts.append({'id_experimento': id_experimento, 'grupo': grupo, 'metrica': f'lift_{metrica_base}', 'valor': lift_valor, 'timestamp': timestamp, 'arquivo_origem': arquivo_origem})
    
    # --- Adicione novas métricas de lift aqui! --- 
    # Se você adicionou 'valor_total' acima, pode adicionar o lift aqui:
    # if 'valor_total' in df_metricas_basicas['metrica'].values:
    #     base_total = df_gc[df_gc['metrica'] == 'valor_total']['valor'].iloc[0]
    #     if base_total > 0:
    #         for grupo in df_metricas_basicas['grupo'].unique():
    #             if grupo == grupo_controle: continue
    #             val_total = df_metricas_basicas[(df_metricas_basicas['grupo'] == grupo) & (df_metricas_basicas['metrica'] == 'valor_total')]['valor'].iloc[0]
    #             lifts.append({'id_experimento': id_experimento, 'grupo': grupo, 'metrica': 'lift_valor_total', 'valor': round((val_total - base_total) / base_total * 100, 2), 'timestamp': timestamp, 'arquivo_origem': arquivo_origem})

    return lifts

def padronizar_grupo_controle(df: pd.DataFrame, grupo_controle_config: str) -> pd.DataFrame:
    """Padroniza o nome do grupo de controle no DataFrame para o valor do config."""
    variacoes_possiveis = {
        grupo_controle_config.lower().replace('_', '').replace(' ', ''),
        "gc", "grupocontrole", "grupo_controle", "grupo controle", "grupo de controle"
    }
    df['grupo'] = df['grupo'].apply(
        lambda x: grupo_controle_config if str(x).strip().lower().replace('_', '').replace(' ', '') in variacoes_possiveis else x
    )
    return df

print("✅ Funções de cálculo de métricas definidas.")

print("Funções de cálculo de métricas definidas.")

✅ Funções de cálculo de métricas definidas.
Funções de cálculo de métricas definidas.


In [14]:
# --- Processamento Principal ---
arquivos_validos = validacao.get('arquivos_validos', [])
print(f"\n🔄 Processando {len(arquivos_validos)} arquivos válidos")

if len(arquivos_validos) == 0:
    print("⚠️ Nenhum arquivo válido para processar")
    df_final = pd.DataFrame(columns=[
        'id_experimento', 'grupo', 'metrica', 'valor', 'timestamp', 'arquivo_origem'
    ])
else:
    todas_metricas = []
    for arquivo_info in arquivos_validos:
        arquivo_key = arquivo_info['key']
        nome_arquivo = arquivo_info['nome']
        print(f"\n📁 Processando {nome_arquivo}...")
        # Ler arquivo do S3 (sem try/except, para que erros de leitura sejam explícitos)
        df = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=arquivo_key)['Body'])
        # Padronizar o nome do grupo de controle no DataFrame
        df = padronizar_grupo_controle(df, grupo_controle)
        # Processar cada experimento dentro do arquivo
        for id_exp in df['id_experimento'].unique():
            df_exp = df[df['id_experimento'] == id_exp]
            # Calcular métricas básicas para cada grupo
            metricas_basicas_exp = []
            for grupo in df_exp['grupo'].unique():
                df_grupo = df_exp[df_exp['grupo'] == grupo]
                metricas_basicas_exp.extend(calcular_metricas_basicas(df_grupo, id_exp, grupo, nome_arquivo))
            df_metricas_basicas_exp = pd.DataFrame(metricas_basicas_exp)
            # Calcular lifts
            lifts_exp = calcular_lifts(df_metricas_basicas_exp, id_exp, grupo_controle, nome_arquivo)
            # Combinar e adicionar à lista geral
            todas_metricas.extend(metricas_basicas_exp)
            todas_metricas.extend(lifts_exp)
            print(f"  ✓ Experimento {id_exp}: {len(metricas_basicas_exp)} métricas básicas + {len(lifts_exp)} lifts")
    # Combinar todas as métricas em um DataFrame final
    if todas_metricas:
        df_final = pd.DataFrame(todas_metricas)
        print(f"\n🎉 Total de métricas calculadas: {len(df_final)}")
    else:
        print("\n⚠️ Nenhuma métrica foi calculada")
        df_final = pd.DataFrame(columns=[
            'id_experimento', 'grupo', 'metrica', 'valor', 'timestamp', 'arquivo_origem'
        ])


🔄 Processando 0 arquivos válidos
⚠️ Nenhum arquivo válido para processar


In [15]:
# Salvar e mostrar resultados
df_final.to_csv(output_metricas_path, index=False)
print(f"\n💾 Métricas salvas em {output_metricas_path}")

# Mostrar resumo detalhado
if len(df_final) > 0:
    print(f"\n📊 Resumo das métricas calculadas:")
    print(f"  • Experimentos: {df_final['id_experimento'].nunique()}")
    print(f"  • Grupos: {df_final['grupo'].nunique()}")
    print(f"  • Tipos de métrica: {df_final['metrica'].nunique()}")
    print(f"  • Total de registros: {len(df_final)}")
    
    print(f"\n📈 Métricas por tipo:")
    for metrica, count in df_final['metrica'].value_counts().items():
        print(f"  • {metrica}: {count} registros")
    
    print(f"\n🔍 Primeiras 5 linhas:")
    print(df_final.head())
else:
    print("\n⚠️ Nenhuma métrica foi gerada")


💾 Métricas salvas em metricas_completas.csv

⚠️ Nenhuma métrica foi gerada


In [16]:
# Upload do resultado para S3
if pasta_processed:
    processed_path = f"{pasta_processed}{output_metricas_path}"
    s3.upload_file(output_metricas_path, bucket_name, processed_path)
    print(f"Resultado em s3://{bucket_name}/{processed_path}")

Resultado em s3://experimento-lucas-barbosa/processed/metricas_completas.csv
